<a href="https://colab.research.google.com/github/SunkaraboinaPraveenKumar/Machine_Learning_Projects/blob/main/RAG_DeepSeek_R1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install pdfplumber langchain langchain_community langchain_core langchain_experimental faiss-cpu

In [7]:
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains import RetrievalQA

In [8]:
loader = PDFPlumberLoader("/content/R22B.Tech.RevisedAcademicRegulations.pdf")
docs=loader.load()
docs

[Document(metadata={'source': '/content/R22B.Tech.RevisedAcademicRegulations.pdf', 'file_path': '/content/R22B.Tech.RevisedAcademicRegulations.pdf', 'page': 0, 'total_pages': 25, 'Author': 'DAP D7', 'CreationDate': "D:20230222112512+05'30'", 'ModDate': "D:20230222112512+05'30'", 'Producer': 'Microsoft: Print To PDF', 'Title': 'Microsoft Word - R22 B.Tech Academic Regulations Modified on 27-01-2023'}, page_content='JAWAHARLAL NEHRU TECHNOLOGICAL UNIVERSITY HYDERABAD\n(Established by State Act No. 30 of 2008)\nKukatpally, Hyderabad, Telangana (India).\nACADEMIC REGULATIONS (R22) FOR B.TECH REGULAR STUDENTS\nWITH EFFECT FROM THE ACADEMIC YEAR 2022-23\n1.0 Under-Graduate Degree Programme in Engineering & Technology (UGP in E&T)\nJawaharlal Nehru Technological University Hyderabad (JNTUH) offers a 4-year (8\nsemesters) Bachelor of Technology (B.Tech.) degree programme, under Choice Based\nCredit System (CBCS) at its non-autonomous constituent and affiliated colleges with\neffect from the ac

In [9]:
text_splitter = SemanticChunker(HuggingFaceEmbeddings())
documents = text_splitter.split_documents(docs)

<ipython-input-9-b819d188c7c3>:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  text_splitter = SemanticChunker(HuggingFaceEmbeddings())


In [10]:
embedder = HuggingFaceEmbeddings()

<ipython-input-10-55e52f88ce51>:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedder = HuggingFaceEmbeddings()


In [11]:
vector = FAISS.from_documents(documents,embedder)
retriever = vector.as_retriever(search_type="similarity",search_kwargs={"k":3})

In [16]:
!pip install langchain_groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.7/412.7 kB 15.4 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.32
    Uninstalling langchain-core-0.3.32:
      Successfully uninstalled langchain-core-0.3.32


In [12]:
from google.colab import userdata
import os
os.environ["GROQ_API_KEY"]=userdata.get('groq_api_key')

In [17]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    temperature=0,
    model_name="deepseek-r1-distill-llama-70b"
)

In [19]:
llm.invoke("What is DeepSeek?").content

'<think>\n\n</think>\n\nDeepSeek Artificial Intelligence Co., Ltd. (referred to as "DeepSeek" or "深度求索") , founded in 2023, is a Chinese company dedicated to making AGI a reality.'

In [29]:
# Define Prompt
prompt="""
1. Use the following piece of context to answer the question at the end.
2. If you donot know the answer, just say that "I don't Know but donot generate by own".
3. Keep answer crisp and limited to 3,4 sentences

Context : {context}
Question: {question}

Helpful Answer:
"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(prompt)

In [30]:
llm_chain = LLMChain(
    llm=llm,
    prompt=QA_CHAIN_PROMPT,
    callbacks=None,
    verbose=True
)

In [31]:
document_prompt = PromptTemplate(
    input_variables=['page_content','source'],
    template="Context : \ncontent:{page_content}\nsource:{source}"
)

In [32]:
combine_documents_chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name='context',
    document_prompt=document_prompt,
    callbacks=None,
)

In [33]:
qa=RetrievalQA(combine_documents_chain=combine_documents_chain,
            verbose=True,
            retriever=retriever,
            return_source_documents=True
            )

In [34]:
user_input = "What is Grading Procedure?"
response = qa(user_input)["result"]
print(response)



> Entering new RetrievalQA chain...


> Entering new LLMChain chain...
Prompt after formatting:

1. Use the following piece of context to answer the question at the end.
2. If you donot know the answer, just say that "I don't Know but donot generate by own".
3. Keep answer crisp and limited to 3,4 sentences

Context : Context : 
content:In the event of the student taking this chance, his Continuous Internal Evaluation marks
for 40 and Semester End Examination marks for 60 obtained in the previous attempt
stand cancelled. 9.0 Grading Procedure
9.1 Grades will be awarded to indicate the performance of students in each Theory Subject,
Laboratory/Practicals/ Industry-Oriented Mini Project/Internship/SDC and Project
Stage. Based on the percentage of marks obtained (Continuous Internal Evaluation plus
Semester End Examination, both taken together) as specified in item 8 above, a
corresponding letter grade shall be given. 9.2 As a measure of the performance of a student, a 10-point absolute

In [35]:
user_input = "What is Sigmoid Activation Function?"
response = qa(user_input)["result"]
print(response)



> Entering new RetrievalQA chain...


> Entering new LLMChain chain...
Prompt after formatting:

1. Use the following piece of context to answer the question at the end.
2. If you donot know the answer, just say that "I don't Know but donot generate by own".
3. Keep answer crisp and limited to 3,4 sentences

Context : Context : 
content:where ‘M’ is the total no. of subjects (as specifically required and listed under the
course structure of the parent department) the student has ‘registered’ i.e., from the 1st
semester onwards up to and inclusive of the 8th semester, ‘j’ is the subject indicator
index (takes into account all subjects from 1 to 8 semesters), C is the no.
source:/content/R22B.Tech.RevisedAcademicRegulations.pdf

Context : 
content:(FAIL)
Absent Ab 0
9.3 A student who has obtained an ‘F’ grade in any subject shall be deemed to have ‘failed’
and is required to reappear as a ‘supplementary student’ in the semester end
examination, as and when offered. In such cases, inter